In [1]:
import pandas as pd
import csv
from surprise import Reader, Dataset, SVD, evaluate
import warnings; warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv("dataset/movies_metadata.csv")
credits = pd.read_csv("dataset/credits.csv")
keywords = pd.read_csv("dataset/keywords.csv")
data = data.merge(credits, on='id')
data = data.merge(keywords, on='id')
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539.0,"[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{'credit_id': '52fe420dc3a36847f800011b', 'de...","[{'id': 612, 'name': 'hotel'}, {'id': 613, 'na..."
1,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,6,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.4,79.0,"[{'cast_id': 7, 'character': 'Frank Wyatt', 'c...","[{'credit_id': '52fe420dc3a36847f800023d', 'de...","[{'id': 520, 'name': 'chicago'}, {'id': 2231, ..."
2,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,11,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.1,6778.0,"[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de...","[{'id': 803, 'name': 'android'}, {'id': 4270, ..."
3,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://movies.disney.com/finding-nemo,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7.6,6292.0,"[{'cast_id': 8, 'character': 'Marlin (voice)',...","[{'credit_id': '52fe420ec3a36847f80006b1', 'de...","[{'id': 494, 'name': 'father son relationship'..."
4,False,NaN,55000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,13,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The world will never be the same, once you've ...",Forrest Gump,False,8.2,8147.0,"[{'cast_id': 7, 'character': 'Forrest Gump', '...","[{'credit_id': '52fe420ec3a36847f800072d', 'de...","[{'id': 422, 'name': 'vietnam veteran'}, {'id'..."


In [3]:
def get_genres(x):
    genre = []
    for i in x:
        genre.append(i['name'])
    return genre

In [4]:
def get_cast(x):
    cast = []
    num = 0
    for i in x:
        i['name'] = i['name'].replace(" ","")
        cast.append(i['name'])
        num = num + 1
        if num==5:
            break
    return cast

In [5]:
def get_crew(x):
    crew = []
    for i in x:
        if i['job'] == 'Director' or i['job'] =='Screenplay':
            i['name'] = i['name'].replace(" ","")
            crew.append(i['name'])
    return crew

In [6]:
def get_keywords(x):
    keywords = []
    for i in x:
        i['name'] = i['name'].replace(" ","")
        keywords.append(i['name'])
    return keywords

In [7]:
from ast import literal_eval

data['genres'] = data['genres'].apply(literal_eval).apply(get_genres)
data['cast'] = data['cast'].apply(literal_eval).apply(get_cast)
data['crew'] = data['crew'].apply(literal_eval).apply(get_crew)
data['keywords'] = data['keywords'].apply(literal_eval).apply(get_keywords)

data['soup'] = data['genres']+data['cast']+data['crew']+data['keywords']
data['soup'] = data['soup'].apply(lambda x: ' '.join(x))

data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,soup
0,False,NaN,4000000,"[Crime, Comedy]",NaN,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539.0,"[TimRoth, AntonioBanderas, JenniferBeals, Mado...","[AllisonAnders, AlexandreRockwell, RobertRodri...","[hotel, newyear'seve, witch, bet, hotelroom, s...",Crime Comedy TimRoth AntonioBanderas JenniferB...
1,False,NaN,0,"[Action, Thriller, Crime]",NaN,6,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",...,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.4,79.0,"[EmilioEstevez, CubaGoodingJr., DenisLeary, Je...","[StephenHopkins, LewisColick]","[chicago, drugdealer, boxingmatch, escape, one...",Action Thriller Crime EmilioEstevez CubaGoodin...
2,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[Adventure, Action, Science Fiction]",http://www.starwars.com/films/star-wars-episod...,11,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,...,Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.1,6778.0,"[MarkHamill, HarrisonFord, CarrieFisher, Peter...",[GeorgeLucas],"[android, galaxy, hermit, deathstar, lightsabe...",Adventure Action Science Fiction MarkHamill Ha...
3,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[Animation, Family]",http://movies.disney.com/finding-nemo,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",...,Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7.6,6292.0,"[AlbertBrooks, EllenDeGeneres, AlexanderGould,...","[AndrewStanton, AndrewStanton, BobPeterson, Da...","[fathersonrelationship, harbor, underwater, fi...",Animation Family AlbertBrooks EllenDeGeneres A...
4,False,NaN,55000000,"[Comedy, Drama, Romance]",NaN,13,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,...,Released,"The world will never be the same, once you've ...",Forrest Gump,False,8.2,8147.0,"[TomHanks, RobinWright, GarySinise, MykeltiWil...","[RobertZemeckis, EricRoth]","[vietnamveteran, hippie, mentallydisabled, run...",Comedy Drama Romance TomHanks RobinWright Gary...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count = CountVectorizer(analyzer='word', stop_words='english')
matrix = count.fit_transform(data['soup'])
similarity = cosine_similarity(matrix,matrix)

In [9]:
ratings = pd.read_csv("dataset/ratings.csv")
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1246,5.0
4,1,1968,4.0


In [10]:
reader = Reader()
dataset = Dataset.load_from_df(ratings, reader)
dataset.split(n_folds=5)
svd = SVD()
evaluate(svd, dataset, measures=['RMSE','MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9265
MAE:  0.7259
------------
Fold 2
RMSE: 0.9062
MAE:  0.7110
------------
Fold 3
RMSE: 0.9232
MAE:  0.7151
------------
Fold 4
RMSE: 0.9385
MAE:  0.7196
------------
Fold 5
RMSE: 0.8940
MAE:  0.6934
------------
------------
Mean RMSE: 0.9177
Mean MAE : 0.7130
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9265123736717984,
                             0.9061706882642786,
                             0.9231536943801815,
                             0.9384820928426807,
                             0.8940311861426411],
                            'mae': [0.7258500082553738,
                             0.7110404781430953,
                             0.7151390518354098,
                             0.7195581808955126,
                             0.6934331401318331]})

In [11]:
trainset = dataset.build_full_trainset()
svd.train(trainset)

In [12]:
titles = data[['title','vote_count','vote_average','id']]
index = pd.Series(data.index, index = data['id'])

In [13]:
def get_movie_list(movieId):
    ind = index[movieId]
    score = list(enumerate(similarity[ind]))
    score = sorted(score, key=lambda x: x[1], reverse=True)
    score = score[0:11]
    movie_indices = [i[0] for i in score]
    return movie_indices

In [14]:
def get_recommendation(userId):
    userdata = ratings[ratings['userId'] == userId]
    userdata = userdata[userdata['rating'] >= 2.5]
    if userdata.empty == True:
        return """Insufficient Data Present"""
    
    movie = set()
    for i in userdata['movieId']:
        movies = get_movie_list(i)
        for x in movies:
            movie.add(x)
    for i in userdata['movieId']:
        movie.remove(index[i])
        
    movie_indices = list(movie)
    top = pd.DataFrame(titles.iloc[movie_indices])
    top['prediction'] = top['id'].apply(lambda x: svd.predict(userId, x).est)
    top = top[top['vote_average'] >= 5]
    top = top.sort_values(by=['prediction','vote_average'], ascending=False)
    return top.head(10)

In [15]:
def add_rating(userId, title, rating):
    if rating>5 or rating<=0:
        return """rating should be between 0-5"""
    try:
        ind = data[data['title'] == title].id.item()
        row = [userId, ind, rating]
        ratings.loc[len(ratings)] = row
        with open('dataset/ratings.csv','a') as f:
            writer = csv.writer(f)
            writer.writerow(row)
    except:
        return """No movie found with this name"""

In [16]:
get_recommendation(1)

,title,vote_count,vote_average,id,prediction
733,Rocky IV,984.0,6.6,1374,3.732269
1393,Masculin Féminin,66.0,7.6,4710,3.711550
734,Rocky V,688.0,5.3,1375,3.689470
829,The Wild Child,65.0,7.3,1627,3.639395
857,Day for Night,100.0,8.0,1675,3.616637
1185,Pierrot le Fou,134.0,7.7,2786,3.616637
5415,A Woman Is a Woman,63.0,7.5,31522,3.616637
1731,Band of Outsiders,103.0,7.5,8073,3.616637
418,When Harry Met Sally...,974.0,7.3,639,3.616637
878,The Last Metro,58.0,7.2,1716,3.616637


In [17]:
get_recommendation(2)

,title,vote_count,vote_average,id,prediction
210,Broken Flowers,359.0,6.8,308,3.749011
747,Cutthroat Island,137.0,5.7,1408,3.684554
5860,Source Code,2752.0,7.1,45612,3.531792
1122,Tie Me Up! Tie Me Down!,102.0,7.0,2469,3.483219
101,Edward Scissorhands,3731.0,7.5,162,3.430544
1327,Scream 2,856.0,6.1,4233,3.410942
358,The Man Who Knew Too Much,276.0,7.3,574,3.374671
628,The Lady Vanishes,183.0,7.5,940,3.350124
8839,The White Helmets,56.0,7.9,416211,3.341553
5400,Mr. Nobody,1616.0,7.9,31011,3.341553


In [18]:
get_recommendation(3)

,title,vote_count,vote_average,id,prediction
526,Los Olvidados,70.0,7.9,800,3.612102
210,Broken Flowers,359.0,6.8,308,3.579436
1204,Michael,174.0,5.5,2928,3.450051
193,Strange Days,334.0,6.8,281,3.358476
5166,Crooks in Clover,96.0,7.8,25253,3.324111
590,The Color Purple,345.0,7.7,873,3.324111
5231,Wake in Fright,66.0,7.6,26405,3.324111
8620,They Call Me Jeeg Robot,575.0,7.5,364433,3.324111
3494,This Is England,370.0,7.4,11798,3.324111
418,When Harry Met Sally...,974.0,7.3,639,3.324111


In [19]:
get_recommendation(4)

,title,vote_count,vote_average,id,prediction
185,Batman Begins,7511.0,7.5,272,4.249530
700,Letters from Iwo Jima,553.0,7.2,1251,4.203081
696,The Good Shepherd,342.0,6.3,1247,4.157672
780,Changing Lanes,306.0,5.9,1537,4.129546
101,Edward Scissorhands,3731.0,7.5,162,4.078059
7365,Batman v Superman: Dawn of Justice,7189.0,5.7,209112,4.065254
840,Forces of Nature,141.0,5.1,1641,4.003907
1310,Benny & Joon,220.0,6.9,4104,3.924541
1320,Spellbound,166.0,7.3,4174,3.912551
959,Laura,173.0,7.6,1939,3.904544


In [20]:
get_recommendation(5)    

,title,vote_count,vote_average,id,prediction
1108,Reign Over Me,340.0,7.1,2355,3.974810
284,Human Nature,74.0,6.1,441,3.936578
334,Terminator Salvation,2496.0,5.9,534,3.869685
145,The Terminator,4208.0,7.4,218,3.814614
867,Re-Animator,263.0,7.0,1694,3.807296
2049,Ghost in the Shell,854.0,7.8,9323,3.787249
4939,Children of Heaven,113.0,7.8,21334,3.787249
5136,Hud,69.0,7.4,24748,3.787249
153,Bride of Frankenstein,208.0,7.4,229,3.787249
3466,The Man Who Shot Liberty Valance,272.0,7.4,11697,3.787249


In [21]:
get_recommendation(251)

,title,vote_count,vote_average,id,prediction
240,Batman Returns,1706.0,6.6,364,3.949557
319,Killing Zoe,111.0,6.1,507,3.931632
532,Se7en,5915.0,8.1,807,3.876225
6013,Horrible Bosses,2341.0,6.4,51540,3.807059
5400,Mr. Nobody,1616.0,7.9,31011,3.791249
7319,My Mom Is a Character,117.0,7.7,203217,3.791249
6694,Captain America: The Winter Soldier,5881.0,7.6,100402,3.791249
5710,Batman: Under the Red Hood,459.0,7.6,40662,3.791249
8835,Team Thor,93.0,7.5,413279,3.791249
7396,Marvel One-Shot: Agent Carter,183.0,7.5,211387,3.791249
